#MOBILEBERT MODEL Dataset : Alexa Reviews

In [2]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 20.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 37.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.1 MB/s eta 0:00:00


In [4]:
import transformers
print(transformers.__version__)

4.30.2


In [5]:
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from transformers import MobileBertTokenizer, MobileBertForSequenceClassification, AdamW
print(torch.__version__)

2.0.1+cu118


In [6]:
# Load the dataset
try:
    data = pd.read_csv('/content/alexa reviews.csv', encoding='ISO-8859-1')
except UnicodeDecodeError as e:
    print(f"Error: {e}")
data.head()




,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [8]:
reviews = data['verified_reviews'].values
labels = data['feedback'].values


In [9]:
# Split the dataset into training and testing sets
train_reviews, test_reviews, train_labels, test_labels = train_test_split(reviews, labels, test_size=0.2, random_state=42)

In [10]:
# Load MobileBERT tokenizer and model
tokenizer = MobileBertTokenizer.from_pretrained('google/mobilebert-uncased')
model = MobileBertForSequenceClassification.from_pretrained('google/mobilebert-uncased')

Some weights of the model checkpoint at google/mobilebert-uncased were not used when initializing MobileBertForSequenceClassification: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing MobileBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some 

In [11]:
# Tokenize and encode the training reviews
train_encodings = tokenizer(list(train_reviews), truncation=True, padding=True, max_length=512, return_tensors='pt')
train_labels = torch.tensor(train_labels)

In [12]:
# Tokenize and encode the testing reviews
test_encodings = tokenizer(list(test_reviews), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_labels = torch.tensor(test_labels)

# Set up the optimizer and the model
optimizer = AdamW(model.parameters(), lr=1e-5)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [13]:
import torch
from torch.utils.data import DataLoader, TensorDataset

In [ ]:

# Set the number of gradient accumulation steps
gradient_accumulation_steps = 2
batch_size = 5

# Train the model
model.train()
total_loss = 0.0

train_dataset = torch.utils.data.TensorDataset(train_encodings['input_ids'], train_encodings['attention_mask'], train_labels)
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

for step, batch in enumerate(train_dataloader):
    # Forward pass
    inputs = batch[0]  # input_ids
    attention_mask = batch[1]
    labels = batch[2]

    outputs = model(input_ids=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss

    # Accumulate gradients
    loss = loss / gradient_accumulation_steps
    total_loss += loss.item()
    loss.backward()

    # Perform the optimizer step after `gradient_accumulation_steps` batches
    if (step + 1) % gradient_accumulation_steps == 0:
        optimizer.step()
        optimizer.zero_grad()



In [ ]:
# Evaluate the model
# Tokenize and encode the testing reviews (assuming you have already done this)
test_encodings = tokenizer(list(test_reviews), truncation=True, padding=True, max_length=512, return_tensors='pt')
test_labels = torch.tensor(test_labels)

# Create a testing dataset and dataloader
test_dataset = torch.utils.data.TensorDataset(test_encodings['input_ids'], test_encodings['attention_mask'], test_labels)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Put the model in evaluation mode
model.eval()

# Initialize variables to store predictions and true labels
all_predictions = []
all_true_labels = []

# Evaluate the model on the testing dataset
with torch.no_grad():
    for batch in test_dataloader:
        inputs = batch[0]  # input_ids
        attention_mask = batch[1]
        labels = batch[2]

        outputs = model(input_ids=inputs, attention_mask=attention_mask)
        logits = outputs.logits

        # Get predicted labels
        predictions = torch.argmax(logits, dim=1).tolist()

        # Collect predictions and true labels
        all_predictions.extend(predictions)
        all_true_labels.extend(labels.tolist())

# Calculate evaluation metrics (e.g., accuracy, precision, recall, F1-score)
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

accuracy = accuracy_score(all_true_labels, all_predictions)
precision, recall, f1, _ = precision_recall_fscore_support(all_true_labels, all_predictions, average='weighted')

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-score: {f1}")


In [ ]:

#ERROR CODE (OUT OF MEMORY ISSUE)
# Train the model
model.train()
optimizer.zero_grad()
outputs = model(input_ids=train_encodings['input_ids'], attention_mask=train_encodings['attention_mask'], labels=train_labels)
loss = outputs.loss
loss.backward()
optimizer.step()
#model.eval()
with torch.no_grad():
    outputs = model(input_ids=test_encodings['input_ids'], attention_mask=test_encodings['attention_mask'])
    logits = outputs.logits
    predictions = torch.argmax(logits, dim=1)
    # Convert predictions and labels to numpy arrays
predictions = predictions.detach().numpy()
test_labels = test_labels.numpy()
# Calculate evaluation metrics
accuracy = accuracy_score(test_labels, predictions)
precision = precision_score(test_labels, predictions)
recall = recall_score(test_labels, predictions)
f1 = f1_score(test_labels, predictions)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)
